In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os 

In [9]:
from DLtools.Trial_evaluation_rec import record_list_result
from DLtools.Data import instant_data,station_sel
from DLtools.feature_sel import call_mar

In [4]:
var = pd.read_csv('/home/song/Public/Song/Work/Thesis/CPY012/4Yr_flood/VAR_opt/VARoutput.csv',index_col=0)
var.reset_index(inplace=True,drop=True)
for i in range(1,72):
    var[str(i)]=var[str(i)].shift(i)
var.columns = [str(i)+'h' for i in range(1,73)]


In [30]:
split_date = '2016-11-01'

loading = instant_data()
df,mode = loading.hourly_instant(),'hour'
# df,mode = loading.daily_instant(),'day'
if mode =='hour': n_past,n_future = 24*6,72
elif mode =='day': n_past,n_future = 60,30

df = df[start_p:stop_p]
data = df
data = data.interpolate(limit=300000000,limit_direction='both').astype('float32')#interpolate neighbor first, for rest NA fill with mean() #.apply(lambda x: x.fillna(x.mean()),axis=0)

st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)

In [35]:
data = data[target]
train,test = data[:split_date],data[split_date:]

In [36]:
val=pd.DataFrame()
for ts in range(72):
    val=pd.concat([val,test.shift(ts)],axis=1)
val.columns = ['{}h'.format(i+1) for i in range(72)]
val

,1h,2h,3h,4h,5h,6h,7h,8h,9h,10h,...,63h,64h,65h,66h,67h,68h,69h,70h,71h,72h
2016-11-01 00:00:00,2.233333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-01 01:00:00,2.195000,2.233333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-01 02:00:00,2.150000,2.195000,2.233333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-01 03:00:00,2.103333,2.150000,2.195000,2.233333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-01 04:00:00,2.055000,2.103333,2.150000,2.195000,2.233333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-01 19:00:00,2.375000,2.375000,2.375000,2.375000,2.375000,2.375,2.375,2.375,2.375,2.375,...,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375
2017-10-01 20:00:00,2.375000,2.375000,2.375000,2.375000,2.375000,2.375,2.375,2.375,2.375,2.375,...,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375
2017-10-01 21:00:00,2.375000,2.375000,2.375000,2.375000,2.375000,2.375,2.375,2.375,2.375,2.375,...,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375
2017-10-01 22:00:00,2.375000,2.375000,2.375000,2.375000,2.375000,2.375,2.375,2.375,2.375,2.375,...,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375,2.375


In [38]:
var.set_index(val.index,inplace=True)

# Load Data

In [8]:
from DLtools.Data import instant_data,station_sel
from DLtools.feature_sel import call_mar
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)
    return df

In [55]:
loading = instant_data()
df,mode = loading.hourly_instant(),'hour'
# df,mode = loading.daily_instant(),'day'
if mode =='hour': n_past,n_future = 24*6,72
elif mode =='day': n_past,n_future = 60,30
st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)

save_path = host_path+'VAR_opt'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Evaluation

In [56]:
def plotgraph(target,save_path,testY,testPredict,syn):
    #mse,nse,r2,rmse,mae = real_eva_error(trainY, trainPredict,)
    Tmse,Tnse,Tr2,Trmse,mae = real_eva_error(testY, testPredict,)

    fig,ax = plt.subplots(1,figsize=(15,5))
    # plt.plot(trainPredict.index,trainPredict, label = "Predict_train",color='g')
    #trainPredict.plot(label = "Predict_train",color='g',lw=1)
    testPredict.plot(label = "Predict_test",color='r',lw=1)
    #ax.scatter(x=trainY.index,y=trainY,marker='.',label='Actual_train',alpha=0.3,edgecolor='lightskyblue',facecolor='None')
    ax.scatter(x=testY.index,y=testY,marker='.',label='Actual_test',alpha=0.3,edgecolor='orange',facecolor='None')

    ax.set_title('[{}]\n'.format(syn)+'Water Level {} Forecast vs Actuals\n'.format(target)+'\nTest  MSE: %.3f | NSE: %.3f | R2 score: %.3f' % (Tmse,Tnse,Tr2))
    ax.legend(loc='upper left', fontsize=8)
    fig.savefig(save_path+'/Plot_{}.png'.format(syn), dpi=300, bbox_inches='tight') 
    fig.clf()
    fig.clear()
    plt.close(fig)

#plt.figure(figsize=(20,5))
#plt.plot(y,label='True')
#plt.plot(yhat,label='Yhat')
#plt.legend()

In [57]:
from DLtools.Trial_evaluation_rec import real_eva_error,plot_rsquare,monsoon_scope
def monsoon_cal(testY,testPredict,syn):        
    m_testPredict = monsoon_scope(testPredict)
    m_test = monsoon_scope(testY)

    Tmse,Tnse,Tr2,Trmse,Tmae = real_eva_error(m_test, m_testPredict,)
    ###### CSV output######
    dict_data = {'Model_':syn,'MSE_trian*':'none','Rmse_trian*':'none','NSE_train*':'none','R2_train*':'none','MAE_train*':'none',
                 'MSE_test*':Tmse,'Rmse_test*':Trmse,'NSE_test*':Tnse,'R2_test*':Tr2,'MAE_test':Tmae}   
    _df = pd.DataFrame.from_dict(data=dict_data, orient ='index')
    return _df

In [70]:
def evaluation(df):
    error=pd.DataFrame()
    for i,col in enumerate(df.columns):  
    #    real_eva_error(arima[col].dropna().values,val[col].dropna.values)
        syn='VAR'
        y = val[col].dropna()
        yhat = df[col].dropna()
        mse,nse,r2,rmse,mae = real_eva_error(yhat,y)
        dict_data = {'Model':syn,'Date':i+1,'Feature':'non','Time_in':n_past,'Time_out':n_future,'Use time':'non',
                    'MSE_trian':'none','Rmse_trian':'none','NSE_train':'none','R2_train':'none','MAE_train':'none',
                     'MSE_test':mse,'Rmse_test':rmse,'NSE_test':nse,'R2_test':r2,'MAE_test':mae}
        _df = pd.DataFrame.from_dict(data=dict_data, orient ='index')

        if i in [0,11,23,47,71]:
            plot_rsquare(save_path,y,yhat,syn+str(i+1))
            plotgraph(target,save_path,y,yhat,syn+str(i+1))
        mon_df = monsoon_cal(y,yhat,syn)
        _df = pd.concat([_df,mon_df])

        error = pd.concat([error,_df],axis=1)
    error.T.to_csv(save_path+'/eval_var.csv',index=False)
    return error.T

In [71]:
var_res = evaluation(var)

In [69]:
var_res[['MSE_test','Rmse_test','NSE_test','R2_test',]]

,MSE_test,Rmse_test,NSE_test,R2_test
0,0.00343308,0.0585925,0.993917,0.993951
0,0.0125367,0.111967,0.977681,0.977944
0,0.0369097,0.192119,0.933993,0.935599
0,0.0706775,0.265852,0.873092,0.878297
0,0.107503,0.327877,0.806268,0.817683
...,...,...,...,...
0,0.15704,0.396283,0.694986,0.731482
0,0.143994,0.379466,0.720318,0.752323
0,0.123927,0.352033,0.759326,0.784976
0,0.0984358,0.313745,0.80885,0.827455


In [53]:
var_res.describe()

,Model,Date,Feature,Time_in,Time_out,Use time,MSE_trian,Rmse_trian,NSE_train,R2_train,...,MSE_trian*,Rmse_trian*,NSE_train*,R2_train*,MAE_train*,MSE_test*,Rmse_test*,NSE_test*,R2_test*,MAE_test
count,72,72,72,72,72,72,72,72,72,72,...,72,72,72,72,72,72.000000,72.000000,72.00000,72.000000,72.0000
unique,1,72,1,1,1,1,1,1,1,1,...,1,1,1,1,1,72.000000,72.000000,72.00000,72.000000,72.0000
top,var,72,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.037531,0.125758,0.44442,0.464416,0.0993
freq,72,1,72,72,72,72,72,72,72,72,...,72,72,72,72,72,1.000000,1.000000,1.00000,1.000000,1.0000


In [143]:
error=pd.DataFrame()
for i,col in enumerate(arima.columns):  
#    real_eva_error(arima[col].dropna().values,val[col].dropna.values)
    syn='var'
    y = val[col].dropna()
    yhat = var[col].dropna()
    mse,nse,r2,rmse,mae = real_eva_error(yhat,y)
    dict_data = {'Model':syn,'Date':i+1,'Feature':'non','Time_in':n_past,'Time_out':n_future,'Use time':'non',
                'MSE_trian':'none','Rmse_trian':'none','NSE_train':'none','R2_train':'none','MAE_train':'none',
                 'MSE_test':mse,'Rmse_test':rmse,'NSE_test':nse,'R2_test':r2,'MAE_test':mae}
    _df = pd.DataFrame.from_dict(data=dict_data, orient ='index')
    
    if i in [0,11,23,47,71]:
        plot_rsquare(save_path,y,yhat,syn+str(i))
        plotgraph(target,save_path,y,yhat,syn+str(i+1))
    mon_df = monsoon_cal(y,yhat,syn)
    _df = pd.concat([_df,mon_df])
    
    error = pd.concat([error,_df],axis=1)
error.T.to_csv(save_path+'/eval_var.csv')

In [126]:
error.T

,Model,Date,Feature,Time_in,Time_out,Use time,MSE_trian,Rmse_trian,NSE_train,R2_train,...,MSE_trian*,Rmse_trian*,NSE_train*,R2_train*,MAE_train*,MSE_test*,Rmse_test*,NSE_test*,R2_test*,MAE_test
0,var,1,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.00405263,0.0636603,0.936227,0.939197,0.0394847
0,var,2,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.00910423,0.0954161,0.856735,0.875056,0.0700418
0,var,3,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0164767,0.128362,0.740721,0.793369,0.101984
0,var,4,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0264885,0.162753,0.583175,0.686108,0.133306
0,var,5,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0382897,0.195678,0.397469,0.560502,0.163492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,var,68,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0802453,0.283276,-0.262748,0.228348,0.24087
0,var,69,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0803844,0.283521,-0.264936,0.228382,0.240968
0,var,70,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0803678,0.283492,-0.264675,0.229409,0.240751
0,var,71,non,144,72,non,none,none,none,none,...,none,none,none,none,none,0.0803105,0.283391,-0.263773,0.230379,0.240468
